In [4]:
import os
import json
import string
import collections

def Process_Descriptions(descriptions):
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for key, desc_list in descriptions.items():
        for i in range(len(desc_list)):
            desc = desc_list[i]
            # tokenize
            desc = desc.split()
            # convert to lower case
            desc = [word.lower() for word in desc]
            # remove punctuation from each token
            desc = [w.translate(table) for w in desc]
            # remove hanging 's' and 'a'
            desc = [word for word in desc if len(word)>1]
            # remove tokens with numbers in them
            desc = [word for word in desc if word.isalpha()]
            # store as string
            desc_list[i] =  ' '.join(desc)           

def save_descriptions(descriptions, filename):
	lines = list()
	for key, desc_list in descriptions.items():
		for desc in desc_list:
			lines.append(str(key) + ' ' + desc)
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

PATH=os.path.abspath('/home/user2/Captioning_ImgsBlind')
annotation_t=PATH+'/annotations/train.json'
annotation_v=PATH+'/annotations/val.json'
with open(annotation_t, 'r') as f:
    annotations_t = json.load(f)
    
with open(annotation_v, 'r') as f:
    annotations_v = json.load(f)
train_img= []
val_img= []
descriptions= collections.OrderedDict()
train_descriptions=collections.OrderedDict()
val_descriptions=collections.OrderedDict()

for annot in annotations_t['annotations']:
    caption = annot['caption']
    image_id = annot['image_id']
    filename = ""
    for img in annotations_t['images']:
        if (img['id'] == image_id):
            filename = img['file_name']
            break
    full_image_path = PATH + '/train/'+filename
    image_id_f = filename.split('.')[0]
    if caption != 'Quality issues are too severe to recognize visual content.':
        if image_id_f not in descriptions.keys():
            descriptions[image_id_f] = list()
        if image_id_f not in train_descriptions.keys():
            train_descriptions[image_id_f] = list()
        descriptions[image_id_f].append(caption)
        train_descriptions[image_id_f].append(caption)
        train_img.append(full_image_path)
    
for annot in annotations_v['annotations']:
    caption = annot['caption']
    image_id = annot['image_id']
    filename = ""
    for img in annotations_v['images']:
        if (img['id'] == image_id):
            filename = img['file_name']
            break
    full_image_path = PATH + '/val/'+filename
    image_id_f = filename.split('.')[0]
    if image_id_f not in descriptions.keys():
        descriptions[image_id_f] = list()
    if image_id_f not in val_descriptions.keys():
        val_descriptions[image_id_f] = list()
    descriptions[image_id_f].append(caption)
    val_descriptions[image_id_f].append(caption)
    val_img.append(full_image_path)

Process_Descriptions(descriptions)
#vocabulary = Create_Vocabulary(descriptions)
#print('Original Vocabulary Size: %d' % len(vocabulary))

save_descriptions(descriptions, 'descriptions.txt')

lines = list()
for key in train_descriptions.keys():
    lines.append(key)
data = '\n'.join(lines)
file = open('training_images_names', 'w')
file.write(data)
file.close()

lines = list()
for key in val_descriptions.keys():
    lines.append(key)
data = '\n'.join(lines)
file = open('val_images_names', 'w')
file.write(data)
file.close()

#print('Dataset: %d' % len(namesOfTrainingImages))

#model = InceptionV3(weights='imagenet')
#model_new = Model(model.input, model.layers[-2].output)

#trainImagesPath='/home/user2/Captioning_ImgsBlind/train/'

train_img_set = set(train_img)
val_img_set = set(val_img)

# Call the funtion to encode all the train images
# This will take a while on CPU - Execute this only once
#start = time()
#encoding_train = {}
#for img in train_img:
#    encoding_train[img[len(trainImagesPath):]] = encode(img)
#print("Time taken in seconds =", time()-start)

In [2]:
from os import listdir
from pickle import dump
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model

# extract features from each photo in the directory
def extract_features():
    # load the model
    model = VGG16()
    # re-structure the model
    model.layers.pop()
    model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
    # summarize
    print(model.summary())
    # extract features from each photo
    features = dict()
    f = open('training_images_names', 'r')
    for name in f:
        # load an image from file
        name = name.strip()
        filename = '/home/user2/Captioning_ImgsBlind/train/' + name+'.jpg'
        image = load_img(filename, target_size=(224, 224))
        # convert the image pixels to a numpy array
        image = img_to_array(image)
        # reshape data for the model
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        # prepare the image for the VGG model
        image = preprocess_input(image)
        # get features
        feature = model.predict(image, verbose=0)
        # get image id
        image_id = name
        # store feature
        features[image_id] = feature
        print('>%s' % name)
    f.close()
    f = open('val_images_names', 'r')
    for name in f:
        # load an image from file
        name = name.strip()
        filename = '/home/user2/Captioning_ImgsBlind/val/' + name+'.jpg'
        image = load_img(filename, target_size=(224, 224))
        # convert the image pixels to a numpy array
        image = img_to_array(image)
        # reshape data for the model
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        # prepare the image for the VGG model
        image = preprocess_input(image)
        # get features
        feature = model.predict(image, verbose=0)
        # get image id
        image_id = name
        # store feature
        features[image_id] = feature
        print('>%s' % name)
    f.close()
    return features

# extract features from all images
#directory = 'Flickr8k_Dataset/Flicker8k_Dataset'
features = extract_features()
print('Extracted Features: %d' % len(features))
# save to file
dump(features, open('features.pkl', 'wb'))

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

>VizWiz_train_00000218
>VizWiz_train_00000219
>VizWiz_train_00000220
>VizWiz_train_00000221
>VizWiz_train_00000222
>VizWiz_train_00000223
>VizWiz_train_00000224
>VizWiz_train_00000225
>VizWiz_train_00000226
>VizWiz_train_00000227
>VizWiz_train_00000228
>VizWiz_train_00000229
>VizWiz_train_00000230
>VizWiz_train_00000231
>VizWiz_train_00000232
>VizWiz_train_00000233
>VizWiz_train_00000234
>VizWiz_train_00000235
>VizWiz_train_00000236
>VizWiz_train_00000237
>VizWiz_train_00000238
>VizWiz_train_00000239
>VizWiz_train_00000240
>VizWiz_train_00000241
>VizWiz_train_00000242
>VizWiz_train_00000243
>VizWiz_train_00000244
>VizWiz_train_00000245
>VizWiz_train_00000246
>VizWiz_train_00000247
>VizWiz_train_00000248
>VizWiz_train_00000249
>VizWiz_train_00000250
>VizWiz_train_00000251
>VizWiz_train_00000252
>VizWiz_train_00000253
>VizWiz_train_00000254
>VizWiz_train_00000255
>VizWiz_train_00000256
>VizWiz_train_00000257
>VizWiz_train_00000258
>VizWiz_train_00000259
>VizWiz_train_00000260
>VizWiz_tra

>VizWiz_train_00000581
>VizWiz_train_00000582
>VizWiz_train_00000583
>VizWiz_train_00000584
>VizWiz_train_00000585
>VizWiz_train_00000586
>VizWiz_train_00000587
>VizWiz_train_00000588
>VizWiz_train_00000589
>VizWiz_train_00000590
>VizWiz_train_00000591
>VizWiz_train_00000592
>VizWiz_train_00000593
>VizWiz_train_00000594
>VizWiz_train_00000595
>VizWiz_train_00000596
>VizWiz_train_00000597
>VizWiz_train_00000598
>VizWiz_train_00000599
>VizWiz_train_00000600
>VizWiz_train_00000601
>VizWiz_train_00000602
>VizWiz_train_00000603
>VizWiz_train_00000604
>VizWiz_train_00000605
>VizWiz_train_00000606
>VizWiz_train_00000607
>VizWiz_train_00000608
>VizWiz_train_00000609
>VizWiz_train_00000610
>VizWiz_train_00000611
>VizWiz_train_00000612
>VizWiz_train_00000613
>VizWiz_train_00000614
>VizWiz_train_00000615
>VizWiz_train_00000616
>VizWiz_train_00000617
>VizWiz_train_00000618
>VizWiz_train_00000619
>VizWiz_train_00000620
>VizWiz_train_00000621
>VizWiz_train_00000622
>VizWiz_train_00000623
>VizWiz_tra

>VizWiz_train_00000944
>VizWiz_train_00000945
>VizWiz_train_00000946
>VizWiz_train_00000947
>VizWiz_train_00000948
>VizWiz_train_00000949
>VizWiz_train_00000950
>VizWiz_train_00000951
>VizWiz_train_00000952
>VizWiz_train_00000953
>VizWiz_train_00000954
>VizWiz_train_00000955
>VizWiz_train_00000956
>VizWiz_train_00000957
>VizWiz_train_00000958
>VizWiz_train_00000959
>VizWiz_train_00000960
>VizWiz_train_00000961
>VizWiz_train_00000962
>VizWiz_train_00000963
>VizWiz_train_00000964
>VizWiz_train_00000965
>VizWiz_train_00000966
>VizWiz_train_00000967
>VizWiz_train_00000968
>VizWiz_train_00000969
>VizWiz_train_00000970
>VizWiz_train_00000971
>VizWiz_train_00000972
>VizWiz_train_00000973
>VizWiz_train_00000974
>VizWiz_train_00000975
>VizWiz_train_00000976
>VizWiz_train_00000977
>VizWiz_train_00000978
>VizWiz_train_00000979
>VizWiz_train_00000980
>VizWiz_train_00000981
>VizWiz_train_00000982
>VizWiz_train_00000983
>VizWiz_train_00000984
>VizWiz_train_00000985
>VizWiz_train_00000986
>VizWiz_tra

>VizWiz_train_00001305
>VizWiz_train_00001306
>VizWiz_train_00001307
>VizWiz_train_00001308
>VizWiz_train_00001309
>VizWiz_train_00001310
>VizWiz_train_00001311
>VizWiz_train_00001312
>VizWiz_train_00001313
>VizWiz_train_00001314
>VizWiz_train_00001315
>VizWiz_train_00001316
>VizWiz_train_00001317
>VizWiz_train_00001318
>VizWiz_train_00001319
>VizWiz_train_00001320
>VizWiz_train_00001321
>VizWiz_train_00001322
>VizWiz_train_00001323
>VizWiz_train_00001324
>VizWiz_train_00001325
>VizWiz_train_00001326
>VizWiz_train_00001327
>VizWiz_train_00001328
>VizWiz_train_00001329
>VizWiz_train_00001330
>VizWiz_train_00001331
>VizWiz_train_00001332
>VizWiz_train_00001333
>VizWiz_train_00001334
>VizWiz_train_00001335
>VizWiz_train_00001336
>VizWiz_train_00001337
>VizWiz_train_00001338
>VizWiz_train_00001339
>VizWiz_train_00001340
>VizWiz_train_00001341
>VizWiz_train_00001342
>VizWiz_train_00001343
>VizWiz_train_00001344
>VizWiz_train_00001345
>VizWiz_train_00001346
>VizWiz_train_00001347
>VizWiz_tra

...

>VizWiz_val_00006120
>VizWiz_val_00006121
>VizWiz_val_00006122
>VizWiz_val_00006123
>VizWiz_val_00006124
>VizWiz_val_00006125
>VizWiz_val_00006126
>VizWiz_val_00006127
>VizWiz_val_00006128
>VizWiz_val_00006129
>VizWiz_val_00006130
>VizWiz_val_00006131
>VizWiz_val_00006132
>VizWiz_val_00006133
>VizWiz_val_00006134
>VizWiz_val_00006135
>VizWiz_val_00006136
>VizWiz_val_00006137
>VizWiz_val_00006138
>VizWiz_val_00006139
>VizWiz_val_00006140
>VizWiz_val_00006141
>VizWiz_val_00006142
>VizWiz_val_00006143
>VizWiz_val_00006144
>VizWiz_val_00006145
>VizWiz_val_00006146
>VizWiz_val_00006147
>VizWiz_val_00006148
>VizWiz_val_00006149
>VizWiz_val_00006150
>VizWiz_val_00006151
>VizWiz_val_00006152
>VizWiz_val_00006153
>VizWiz_val_00006154
>VizWiz_val_00006155
>VizWiz_val_00006156
>VizWiz_val_00006157
>VizWiz_val_00006158
>VizWiz_val_00006159
>VizWiz_val_00006160
>VizWiz_val_00006161
>VizWiz_val_00006162
>VizWiz_val_00006163
>VizWiz_val_00006164
>VizWiz_val_00006165
>VizWiz_val_00006166
>VizWiz_val_0

>VizWiz_val_00006511
>VizWiz_val_00006512
>VizWiz_val_00006513
>VizWiz_val_00006514
>VizWiz_val_00006515
>VizWiz_val_00006516
>VizWiz_val_00006517
>VizWiz_val_00006518
>VizWiz_val_00006519
>VizWiz_val_00006520
>VizWiz_val_00006521
>VizWiz_val_00006522
>VizWiz_val_00006523
>VizWiz_val_00006524
>VizWiz_val_00006525
>VizWiz_val_00006526
>VizWiz_val_00006527
>VizWiz_val_00006528
>VizWiz_val_00006529
>VizWiz_val_00006530
>VizWiz_val_00006531
>VizWiz_val_00006532
>VizWiz_val_00006533
>VizWiz_val_00006534
>VizWiz_val_00006535
>VizWiz_val_00006536
>VizWiz_val_00006537
>VizWiz_val_00006538
>VizWiz_val_00006539
>VizWiz_val_00006540
>VizWiz_val_00006541
>VizWiz_val_00006542
>VizWiz_val_00006543
>VizWiz_val_00006544
>VizWiz_val_00006545
>VizWiz_val_00006546
>VizWiz_val_00006547
>VizWiz_val_00006548
>VizWiz_val_00006549
>VizWiz_val_00006550
>VizWiz_val_00006551
>VizWiz_val_00006552
>VizWiz_val_00006553
>VizWiz_val_00006554
>VizWiz_val_00006555
>VizWiz_val_00006556
>VizWiz_val_00006557
>VizWiz_val_0

>VizWiz_val_00006909
>VizWiz_val_00006910
>VizWiz_val_00006911
>VizWiz_val_00006912
>VizWiz_val_00006913
>VizWiz_val_00006914
>VizWiz_val_00006915
>VizWiz_val_00006916
>VizWiz_val_00006917
>VizWiz_val_00006918
>VizWiz_val_00006919
>VizWiz_val_00006920
>VizWiz_val_00006921
>VizWiz_val_00006922
>VizWiz_val_00006923
>VizWiz_val_00006924
>VizWiz_val_00006925
>VizWiz_val_00006926
>VizWiz_val_00006927
>VizWiz_val_00006928
>VizWiz_val_00006929
>VizWiz_val_00006930
>VizWiz_val_00006931
>VizWiz_val_00006932
>VizWiz_val_00006933
>VizWiz_val_00006934
>VizWiz_val_00006935
>VizWiz_val_00006936
>VizWiz_val_00006937
>VizWiz_val_00006938
>VizWiz_val_00006939
>VizWiz_val_00006940
>VizWiz_val_00006941
>VizWiz_val_00006942
>VizWiz_val_00006943
>VizWiz_val_00006944
>VizWiz_val_00006945
>VizWiz_val_00006946
>VizWiz_val_00006947
>VizWiz_val_00006948
>VizWiz_val_00006949
>VizWiz_val_00006950
>VizWiz_val_00006951
>VizWiz_val_00006952
>VizWiz_val_00006953
>VizWiz_val_00006954
>VizWiz_val_00006955
>VizWiz_val_0

>VizWiz_val_00007301
>VizWiz_val_00007302
>VizWiz_val_00007303
>VizWiz_val_00007304
>VizWiz_val_00007305
>VizWiz_val_00007306
>VizWiz_val_00007307
>VizWiz_val_00007308
>VizWiz_val_00007309
>VizWiz_val_00007310
>VizWiz_val_00007311
>VizWiz_val_00007312
>VizWiz_val_00007313
>VizWiz_val_00007314
>VizWiz_val_00007315
>VizWiz_val_00007316
>VizWiz_val_00007317
>VizWiz_val_00007318
>VizWiz_val_00007319
>VizWiz_val_00007320
>VizWiz_val_00007321
>VizWiz_val_00007322
>VizWiz_val_00007323
>VizWiz_val_00007324
>VizWiz_val_00007325
>VizWiz_val_00007326
>VizWiz_val_00007327
>VizWiz_val_00007328
>VizWiz_val_00007329
>VizWiz_val_00007330
>VizWiz_val_00007331
>VizWiz_val_00007332
>VizWiz_val_00007333
>VizWiz_val_00007334
>VizWiz_val_00007335
>VizWiz_val_00007336
>VizWiz_val_00007337
>VizWiz_val_00007338
>VizWiz_val_00007339
>VizWiz_val_00007340
>VizWiz_val_00007341
>VizWiz_val_00007342
>VizWiz_val_00007343
>VizWiz_val_00007344
>VizWiz_val_00007345
>VizWiz_val_00007346
>VizWiz_val_00007347
>VizWiz_val_0

>VizWiz_val_00007698
>VizWiz_val_00007699
>VizWiz_val_00007700
>VizWiz_val_00007701
>VizWiz_val_00007702
>VizWiz_val_00007703
>VizWiz_val_00007704
>VizWiz_val_00007705
>VizWiz_val_00007706
>VizWiz_val_00007707
>VizWiz_val_00007708
>VizWiz_val_00007709
>VizWiz_val_00007710
>VizWiz_val_00007711
>VizWiz_val_00007712
>VizWiz_val_00007713
>VizWiz_val_00007714
>VizWiz_val_00007715
>VizWiz_val_00007716
>VizWiz_val_00007717
>VizWiz_val_00007718
>VizWiz_val_00007719
>VizWiz_val_00007720
>VizWiz_val_00007721
>VizWiz_val_00007722
>VizWiz_val_00007723
>VizWiz_val_00007724
>VizWiz_val_00007725
>VizWiz_val_00007726
>VizWiz_val_00007727
>VizWiz_val_00007728
>VizWiz_val_00007729
>VizWiz_val_00007730
>VizWiz_val_00007731
>VizWiz_val_00007732
>VizWiz_val_00007733
>VizWiz_val_00007734
>VizWiz_val_00007735
>VizWiz_val_00007736
>VizWiz_val_00007737
>VizWiz_val_00007738
>VizWiz_val_00007739
>VizWiz_val_00007740
>VizWiz_val_00007741
>VizWiz_val_00007742
>VizWiz_val_00007743
>VizWiz_val_00007744
>VizWiz_val_0

In [ ]:
from numpy import array
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint
import keras
import numpy as np

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line
		dataset.append(identifier)
	return set(dataset)

# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions

# load photo features
def load_photo_features(filename, dataset):
	# load all features
	all_features = load(open(filename, 'rb'))
	# filter features
	features = {k: all_features[k] for k in dataset}
	return features

# covert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
	lines = to_lines(descriptions)
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# calculate the length of the description with the most words
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)

# create sequences of images, input sequences and output words for an image
def create_sequences(tokenizer, max_length, desc_list, photo, vocab_size):
	X1, X2, y = list(), list(), list()
	# walk through each description for the image
	for desc in desc_list:
		# encode the sequence
		seq = tokenizer.texts_to_sequences([desc])[0]
		# split one sequence into multiple X,y pairs
		for i in range(1, len(seq)):
			# split into input and output pair
			in_seq, out_seq = seq[:i], seq[i]
			# pad input sequence
			in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
			# encode output sequence
			out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
			# store
			X1.append(photo)
			X2.append(in_seq)
			y.append(out_seq)
	return array(X1), array(X2), array(y)

# define the captioning model
def define_model(vocab_size, max_length):
	# feature extractor model
	inputs1 = Input(shape=(4096,))
	fe1 = Dropout(0.5)(inputs1)
	fe2 = Dense(256, activation='relu')(fe1)
	# sequence model
	inputs2 = Input(shape=(max_length,))
	se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
	se2 = Dropout(0.5)(se1)
	se3 = LSTM(256)(se2)
	# decoder model
	decoder1 = add([fe2, se3])
	decoder2 = Dense(256, activation='relu')(decoder1)
	outputs = Dense(vocab_size, activation='softmax')(decoder2)
	# tie it together [image, seq] [word]
	model = Model(inputs=[inputs1, inputs2], outputs=outputs)
	# compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam')
	# summarize model
	model.summary()
	#plot_model(model, to_file='model.png', show_shapes=True)
	return model

def data_generator(descriptions, photos, tokenizer, max_length, vocab_size):
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            # retrieve the photo feature
            photo = photos[key][0]
            in_img, in_seq, out_word = create_sequences(tokenizer, max_length, desc_list, photo, vocab_size)
            yield [[in_img, in_seq], out_word]

# data generator, intended to be used in a call to model.fit_generator()
def data_generator1(descriptions, photos, tokenizer, max_length, vocab_size, photos_per_batch):            
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            # retrieve the photo feature
            if key in photos.keys():
                photo = photos[key][0]
                n+=1
                for desc in desc_list:
                    # encode the sequence
                    seq = tokenizer.texts_to_sequences([desc])[0]
                    # split one sequence into multiple X, y pairs
                    for i in range(1, len(seq)):
                        # split into input and output pair
                        in_seq, out_seq = seq[:i], seq[i]
                        # pad input sequence
                        in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                        # encode output sequence
                        out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                        # store
                        X1.append(photo)
                        X2.append(in_seq)
                        y.append(out_seq)

            # yield the batch data
            if n==photos_per_batch:
                X11=np.reshape(array(X1),(array(X1).shape[0],4096))
                yield [[X11, array(X2)], array(y)]
                #yield [ X11.shape]                
                X1, X2, y = list(), list(), list()
                n=0

filename = 'training_images_names'
train = load_set(filename)
print('Dataset: %d' % len(train))
# descriptions
train_descriptions = load_clean_descriptions('descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))
# photo features
train_features = load_photo_features('features.pkl', train)
print('Photos: train=%d' % len(train_features))
# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)

# define the model
model = define_model(vocab_size, max_length)
# train the model, run epochs manually and save after each epoch
epochs = 6
steps = len(train_descriptions)
photos_per_batch = 0
for i in range(epochs):
    # create the data generator
    if i<2:
        keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
        photos_per_batch = 6
    elif i<4:
        keras.optimizers.Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999, amsgrad=False)
        photos_per_batch = 12
    else:
        keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
        photos_per_batch = 24
    generator = data_generator1(train_descriptions, train_features, tokenizer, max_length, vocab_size, photos_per_batch)
    # fit for one epoch
    steps = len(train_descriptions)//photos_per_batch
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1, initial_epoch=0)
    # save model
    model.save('model_' + str(i) + '.h5')

Using TensorFlow backend.


Dataset: 22900
Descriptions: train=22900
Photos: train=22900


In [8]:
from numpy import argmax
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)

# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions

# load photo features
def load_photo_features(filename, dataset):
	# load all features
	all_features = load(open(filename, 'rb'))
	# filter features
	features = {k: all_features[k] for k in dataset}
	return features

# covert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
	lines = to_lines(descriptions)
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# calculate the length of the description with the most words
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)

# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
	# seed the generation process
	in_text = 'startseq'
	# iterate over the whole length of the sequence
	for i in range(max_length):
		# integer encode input sequence
		sequence = tokenizer.texts_to_sequences([in_text])[0]
		# pad input
		sequence = pad_sequences([sequence], maxlen=max_length)
		# predict next word
		yhat = model.predict([photo,sequence], verbose=0)
		# convert probability to integer
		yhat = argmax(yhat)
		# map integer to word
		word = word_for_id(yhat, tokenizer)
		# stop if we cannot map the word
		if word is None:
			break
		# append as input for generating the next word
		in_text += ' ' + word
		# stop if we predict the end of the sequence
		if word == 'endseq':
			break
	return in_text

# evaluate the skill of the model
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
	actual, predicted = list(), list()
	# step over the whole set
	for key, desc_list in descriptions.items():
		# generate description
		yhat = generate_desc(model, tokenizer, photos[key], max_length)
		# store actual and predicted
		references = [d.split() for d in desc_list]
		actual.append(references)
		predicted.append(yhat.split())
	# calculate BLEU score
	print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
	print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
	print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
	print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

# prepare tokenizer on train set

# load training dataset (6K)
filename = 'training_images_names'
train = load_set(filename)
print('Dataset: %d' % len(train))
# descriptions
train_descriptions = load_clean_descriptions('descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))
# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)

# prepare test set

# load test set
filename = 'val_images_names'
test = load_set(filename)
print('Dataset: %d' % len(test))
# descriptions
test_descriptions = load_clean_descriptions('descriptions.txt', test)
print('Descriptions: test=%d' % len(test_descriptions))
# photo features
test_features = load_photo_features('features.pkl', test)
print('Photos: test=%d' % len(test_features))

# load the model
filename = 'model_good_1.h5'
model = load_model(filename)
# evaluate model
evaluate_model(model, test_descriptions, test_features, tokenizer, max_length)

Dataset: 22900
Descriptions: train=22900
Vocabulary Size: 18006
Description Length: 105
Dataset: 7750
Descriptions: test=7750
Photos: test=7750


/home/user2/.local/lib/python3.5/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


BLEU-1: 0.545750
BLEU-2: 0.299972
BLEU-3: 0.212191
BLEU-4: 0.104786


In [1]:
from keras.preprocessing.text import Tokenizer
from pickle import dump

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line
		dataset.append(identifier)
	return set(dataset)

# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions

# covert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
	lines = to_lines(descriptions)
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# load training dataset (6K)
filename = 'training_images_names'
train = load_set(filename)
print('Dataset: %d' % len(train))
# descriptions
train_descriptions = load_clean_descriptions('descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))
# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

Using TensorFlow backend.


Dataset: 22900
Descriptions: train=22900


In [ ]:
train_img_set

In [ ]:
from pickle import load
from numpy import argmax
from keras.preprocessing.sequence import pad_sequences
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.models import load_model
import matplotlib.pyplot as plt

# extract features from each photo in the directory
def extract_features(filename):
	# load the model
	model = VGG16()
	# re-structure the model
	model.layers.pop()
	model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
	# load the photo
	image = load_img(filename, target_size=(224, 224))
	# convert the image pixels to a numpy array
	image = img_to_array(image)
	# reshape data for the model
	image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
	# prepare the image for the VGG model
	image = preprocess_input(image)
	# get features
	feature = model.predict(image, verbose=0)
	return feature

# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
	# seed the generation process
	in_text = 'startseq'
	# iterate over the whole length of the sequence
	for i in range(max_length):
		# integer encode input sequence
		sequence = tokenizer.texts_to_sequences([in_text])[0]
		# pad input
		sequence = pad_sequences([sequence], maxlen=max_length)
		# predict next word
		yhat = model.predict([photo,sequence], verbose=0)
		# convert probability to integer
		yhat = argmax(yhat)
		# map integer to word
		word = word_for_id(yhat, tokenizer)
		# stop if we cannot map the word
		if word is None:
			break
		# append as input for generating the next word
		in_text += ' ' + word
		# stop if we predict the end of the sequence
		if word == 'endseq':
			break
	return in_text

# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))
# pre-define the max sequence length (from training)
max_length = 105
# load the model
model = load_model('model_good_1.h5')
# load and prepare the photograph
path = '/home/user2/Captioning_ImgsBlind/val/VizWiz_val_00000000.jpg'
plt.imshow(plt.imread(path))
photo = extract_features(path)
# generate description
description = generate_desc(model, tokenizer, photo, max_length)
print(description)